In [1]:
import os
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN

# Qurilmani aniqlash
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model yuklash
mtcnn = MTCNN(keep_all=True, device=device)

# Dataset papkasini yaratish
DATASET_PATH = "dataset"
if not os.path.exists(DATASET_PATH):
    os.makedirs(DATASET_PATH)

# Yangi ID yaratish funksiyasi
def get_new_id():
    existing_ids = [int(folder) for folder in os.listdir(DATASET_PATH) if folder.isdigit()]
    return max(existing_ids) + 1 if existing_ids else 1

# Kamera ochish
cap = cv2.VideoCapture(0)

# Foydalanuvchi nomini kiritish
person_name = input("Foydalanuvchi ismini kiriting: ").strip()
person_id = get_new_id()  # Yangi ID yaratish
save_path = os.path.join(DATASET_PATH, str(person_id))

# Rasm saqlash uchun papka yaratish
os.makedirs(save_path, exist_ok=True)

print(f"{person_name} uchun ID: {person_id}")
print(f"Rasmlar {save_path} ichiga saqlanadi.")

frame_count = 0
max_frames = 50  # Har bir shaxs uchun 50 ta rasm

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(rgb_frame)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(b) for b in box]
            face = rgb_frame[y1:y2, x1:x2]

            if face.size == 0:
                continue

            frame_count += 1
            file_name = f"{frame_count}.jpg"
            file_path = os.path.join(save_path, file_name)

            # Rasmni saqlash
            cv2.imwrite(file_path, cv2.cvtColor(face, cv2.COLOR_RGB2BGR))

            # Yuzni belgilash
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Saving {frame_count}/{max_frames}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Face Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

print(f"{person_name} uchun 50 ta rasm saqlandi. ID: {person_id}")

cap.release()
cv2.destroyAllWindows()


Muhammadali uchun ID: 2
Rasmlar dataset\2 ichiga saqlanadi.
Muhammadali uchun 50 ta rasm saqlandi. ID: 2


In [2]:
import os
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image

# Qurilmani aniqlash
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model yuklash
mtcnn = MTCNN(keep_all=False, device=device)  # Faqat bitta yuz kerak bo‘lsa
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Transform funksiyasi
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Embedding olish funksiyasi
def get_embedding(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"[X] Rasm ochilmadi: {image_path}")
        return None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img).convert("RGB")

    try:
        boxes, _ = mtcnn.detect(img_pil)
        if boxes is None or len(boxes) == 0:
            print(f"[!] Yuz topilmadi: {image_path}")
            return None
    except Exception as e:
        print(f"[X] MTCNN.detect xatolik ({image_path}): {e}")
        return None

    x1, y1, x2, y2 = [int(v) for v in boxes[0]]
    h, w, _ = img.shape
    x1, y1, x2, y2 = [max(0, min(p, limit)) for p, limit in zip([x1, y1, x2, y2], [w, h, w, h])]

    face = img[y1:y2, x1:x2]
    if face.size == 0:
        print(f"[!] Bo‘sh yuz maydoni: {image_path}")
        return None

    face_tensor = transform(face).unsqueeze(0).to(device)

    with torch.no_grad():
        embedding = resnet(face_tensor).cpu().numpy()[0]

    return embedding

# Datasetdan yuzlarni embedding qilish
DATASET_PATH = "dataset"
known_faces = {}

for person_id in os.listdir(DATASET_PATH):
    person_path = os.path.join(DATASET_PATH, person_id)
    if os.path.isdir(person_path):
        images = [os.path.join(person_path, img) for img in os.listdir(person_path) if img.endswith(".jpg")]
        embeddings = [get_embedding(img) for img in images]
        embeddings = [e for e in embeddings if e is not None]

        if embeddings:
            known_faces[person_id] = np.mean(embeddings, axis=0)

print(f"{len(known_faces)} ta foydalanuvchi yuklandi.")

# Jonli kameradan yuzni tanish
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(rgb_frame)

    boxes, _ = mtcnn.detect(img_pil)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = [int(v) for v in box]
            h, w, _ = frame.shape
            x1, y1, x2, y2 = [max(0, min(p, limit)) for p, limit in zip([x1, y1, x2, y2], [w, h, w, h])]
            face = rgb_frame[y1:y2, x1:x2]

            if face.size == 0:
                continue

            try:
                face_tensor = transform(face).unsqueeze(0).to(device)
                embedding = resnet(face_tensor).detach().cpu().numpy()

                best_match = "Unknown"
                best_similarity = 0

                for person_id, known_embedding in known_faces.items():
                    similarity = np.dot(embedding, known_embedding.T) / (np.linalg.norm(embedding) * np.linalg.norm(known_embedding))
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match = f"ID: {person_id}"

                if best_similarity > 0.8:
                    text, color = best_match, (0, 255, 0)
                else:
                    text, color = "Unknown", (0, 0, 255)

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            except Exception as e:
                print("Error:", e)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


[!] Yuz topilmadi: dataset\1\1.jpg
[!] Yuz topilmadi: dataset\1\10.jpg
[!] Yuz topilmadi: dataset\1\11.jpg
[!] Yuz topilmadi: dataset\1\12.jpg
[!] Yuz topilmadi: dataset\1\13.jpg
[!] Yuz topilmadi: dataset\1\14.jpg
[!] Yuz topilmadi: dataset\1\17.jpg
[!] Yuz topilmadi: dataset\1\18.jpg
[!] Yuz topilmadi: dataset\1\19.jpg
[!] Yuz topilmadi: dataset\1\2.jpg
[!] Yuz topilmadi: dataset\1\20.jpg
[!] Yuz topilmadi: dataset\1\21.jpg
[!] Yuz topilmadi: dataset\1\22.jpg
[!] Yuz topilmadi: dataset\1\24.jpg
[!] Yuz topilmadi: dataset\1\25.jpg
[!] Yuz topilmadi: dataset\1\26.jpg
[!] Yuz topilmadi: dataset\1\27.jpg
[!] Yuz topilmadi: dataset\1\28.jpg
[!] Yuz topilmadi: dataset\1\29.jpg
[!] Yuz topilmadi: dataset\1\3.jpg
[!] Yuz topilmadi: dataset\1\30.jpg
[!] Yuz topilmadi: dataset\1\31.jpg
[!] Yuz topilmadi: dataset\1\32.jpg
[!] Yuz topilmadi: dataset\1\33.jpg
[!] Yuz topilmadi: dataset\1\34.jpg
[!] Yuz topilmadi: dataset\1\35.jpg
[!] Yuz topilmadi: dataset\1\36.jpg
[!] Yuz topilmadi: dataset\1\37

In [5]:
import torch
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("OpenCV:", cv2.__version__)


Torch: 2.6.0+cpu
CUDA available: False
OpenCV: 4.10.0
